In [ ]:
import autograd.numpy as np
from autograd import grad
from autograd.numpy import linalg
from autograd.numpy.random import multivariate_normal, normal

In [ ]:
#initialise parameters
N = 100 # Number of iterations
M = 5 # dimensionality of state
e = 1e-5 # convergence condition

R = [] # Process noise Covariance
Q = [] # Measurement Noise
z = [] # initial state
A = [] # state transition
B = [] # noise matrix
C = [] # observation matrix

y = []
E = linalg.inv(np.block([[Q,np.zeros(M,M)],[np.zeros([M,M]),R]]))
p = 0.001

In [ ]:
def ml_gradient_desc():
    #initial guesses
    z = normal(0,1,[M,1])
    u = normal(0,1,[M,1])
    step_dir = grad(ml_cost)
    error = 1
    #iterate till desired precision reached
    while error > e:
        #perform gradient descent
        x = step_dir(z,u)
        error = linalg.norm(p*x)
        z = z + p*x[:M] 
        u = u + p*x[M:]
    
    #create next estimate
    z_next = np.dot(A,z) + np.dot(B,u)
    return z_next, z, u

In [ ]:
def ml_cost(z,u):
    x = np.block([u,y-np.dot(C,z)])
    return np.dot(np.dot(x,E),x.T)

In [ ]:
for i in range(N):
    u = multivariate_normal(np.zeros(M),Q).T
    v = multivariate_normal(np.zeros(M),R).T
    z = np.dot(A,z) + np.dot(B,u)
    y = np.dot(C,u) + v
    
    #perform PDMM estimation
    z_new, ml_gradient_desc()
    #perform Kalman estimation
    
    